In [108]:
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import cv2
import numpy as np
import pandas as pd

transform = transforms.Compose([
    transforms.Resize((50,50)),  # 원하는 크기로 이미지 리사이즈
    transforms.ToTensor()  # 이미지를 텐서로 변환
])

# ImageFolder 데이터셋 생성
data = ImageFolder(root="./data", transform=transform)

In [109]:
data_loader = DataLoader(data, batch_size=10, shuffle=True)

In [110]:
images, labels = next(iter(data_loader))
print(images.shape, labels.shape)

torch.Size([10, 3, 50, 50]) torch.Size([10])


In [111]:
# CNN 모델클래스 생성
class Mob_Dog(nn.Module):
    def __init__(self):
        super(Mob_Dog, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [112]:
# 모델 객체 생성, 옵티마이저 생성
model = Mob_Dog()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [117]:
from tqdm import tqdm

# 학습 진행
best_accuracy = 0.0
no_improvement_count = 0
for epoch in range(10):
    running_loss = 0.0
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for i, data in pbar:
        inputs, labels = data
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 10 == 9:
            pbar.set_description(f'Epoch [{epoch + 1}/{10}], Step [{i + 1}/{len(data_loader)}], Loss: {running_loss / 10:.4f}')
            running_loss = 0.0
            
    # 모델 평가
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    epoch_accuracy = correct / total
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * epoch_accuracy))
    
    # Scheduler로 정확도 변화를 관찰하여 학습 조기 종료
    if epoch_accuracy > best_accuracy:
        best_accuracy = epoch_accuracy
        no_improvement_count = 0
    else:
        no_improvement_count += 1
        if no_improvement_count >= 3:
            print(f'No improvement in accuracy for {no_improvement_count} epochs. Training stopped.')
            break

print('Finished Training')


Epoch [1/10], Step [10/18], Loss: 0.6765: 100%|██████████| 18/18 [00:00<00:00, 35.54it/s]


Accuracy of the network on the 10000 test images: 53 %


Epoch [2/10], Step [10/18], Loss: 0.6839: 100%|██████████| 18/18 [00:00<00:00, 49.90it/s]


Accuracy of the network on the 10000 test images: 53 %


Epoch [3/10], Step [10/18], Loss: 0.6845: 100%|██████████| 18/18 [00:00<00:00, 49.91it/s]


Accuracy of the network on the 10000 test images: 53 %


Epoch [4/10], Step [10/18], Loss: 0.6856: 100%|██████████| 18/18 [00:00<00:00, 52.50it/s]


Accuracy of the network on the 10000 test images: 53 %
No improvement in accuracy for 3 epochs. Training stopped.
Finished Training


In [116]:
# 모델 저장
torch.save(model.state_dict(), 'mob_dog.pth')

# 모델 로드
model = Mob_Dog()
model.load_state_dict(torch.load('mob_dog.pth'))
model.eval()

# 이미지 예측
img = cv2.imread('./data/mop_dog/0.jpg')
img = cv2.resize(img, (50, 50))
#img = img / 255.0
img = np.transpose(img, (2, 0, 1))
img = torch.Tensor(img)
img = img.unsqueeze(0)

with torch.no_grad():
    output = model(img)
    _, predicted = torch.max(output.data, 1)
predicted # 이런 젠장, 걸레라고 나온다 

tensor([0])